In [1]:
import json
import os
import time
import random
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import shutil

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets
from torchvision import transforms
from torchvision import models
from torchsummary import summary
from sklearn.model_selection import train_test_split

from model_pytorch import EfficientNet
from utils import Bar,Logger, AverageMeter, accuracy, mkdir_p, savefig
from warmup_scheduler import GradualWarmupScheduler
from sklearn.metrics import accuracy_score

from PIL import ImageFile, ImageOps
ImageFile.LOAD_TRUNCATED_IMAGES = True
import collections

In [2]:
# GPU Device
gpu_id = 3
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_id)
use_cuda = torch.cuda.is_available()
print("GPU device %d:" %(gpu_id), use_cuda)

GPU device 3: True


# Arguments

In [3]:
source_dir = '/media/data2/dataset/GAN_ImageData/StyleGAN_256/'
target_dir = '/media/data2/dataset/GAN_ImageData/PGGAN_128/'

In [4]:
pretrained = './log/style1/128/b1/checkpoint.pth.tar'
resume = ''

In [5]:
# Model
model_name = 'efficientnet-b1' # b0-b7 scale

# Optimization
num_classes = 2
epochs = 4000
start_epoch = 0
train_batch = 100
test_batch = 200
lr = 0.01
schedule = [200, 1500, 3000]
momentum = 0.9
gamma = 0.1 # LR is multiplied by gamma on schedule

# CheckPoint
checkpoint = './log/style1/128/b1/to_pggan/contrastive' # dir
if not os.path.isdir(checkpoint):
    os.makedirs(checkpoint)
num_workers = 8

# Seed
manual_seed = 7
random.seed(manual_seed)
torch.cuda.manual_seed_all(manual_seed)

# Image
size = (128, 128)

# sp
sp_alpha = 0.1
sp_beta = 0.01
fc_name = '_fc.'

# iterative training
feedback = 0
iter_time = [1000, 2000, 2500]

# cutmix
cm_prob = 0.5
cm_prob_init = 0.99
cm_prob_low = 0.01

# constrastive
thresh = 0.4

best_acc = 0

In [6]:
state = {}
state['num_classes'] = num_classes
state['epochs'] = epochs
state['start_epoch'] = start_epoch
state['train_batch'] = train_batch
state['test_batch'] = test_batch
state['lr'] = lr
state['schedule'] = schedule
state['momentum'] = momentum
state['gamma'] = gamma

In [7]:
class SiameseNetworkDataset(Dataset):
    
    def __init__(self,imageFolderDataset,transform=None,should_invert=True):
        self.imageFolderDataset = imageFolderDataset    
        self.transform = transform
        self.should_invert = should_invert
        
    def __getitem__(self,index):
        img0_tuple = random.choice(self.imageFolderDataset.imgs)
        #we need to make sure approx 50% of images are in the same class
        should_get_same_class = random.randint(0,1) 
        if should_get_same_class:
            while True:
                #keep looping till the same class image is found
                img1_tuple = random.choice(self.imageFolderDataset.imgs) 
                if img0_tuple[1]==img1_tuple[1]:
                    break
        else:
            while True:
                #keep looping till a different class image is found
                
                img1_tuple = random.choice(self.imageFolderDataset.imgs) 
                if img0_tuple[1] !=img1_tuple[1]:
                    break

        img0 = Image.open(img0_tuple[0])
        img1 = Image.open(img1_tuple[0])
#         img0 = img0.convert("L")
#         img1 = img1.convert("L")
        
        if self.should_invert:
            img0 = PIL.ImageOps.invert(img0)
            img1 = PIL.ImageOps.invert(img1)

        if self.transform is not None:
            img0 = self.transform(img0)
            img1 = self.transform(img1)
        
        return img0, img1 , torch.from_numpy(np.array([int(img1_tuple[1]!=img0_tuple[1])],dtype=np.float32))
    
    def __len__(self):
        return len(self.imageFolderDataset.imgs)

In [8]:
class ContrastiveLoss(torch.nn.Module):
    """
    Contrastive loss function.
    Based on: http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    """

    def __init__(self, margin=2.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, output1, output2, label):
        euclidean_distance = F.pairwise_distance(output1, output2, keepdim = True)
        loss_contrastive = torch.mean((1-label) * torch.pow(euclidean_distance, 2) +
                                      (label) * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2))


        return loss_contrastive

# Dataset

In [9]:
train_dir = os.path.join(target_dir, '100_shot_style1')
source_train_dir = os.path.join(target_dir, '100_shot_style1')
val_target_dir = os.path.join(target_dir, 'validation')
val_source_dir = os.path.join(source_dir, 'validation')

train_aug = transforms.Compose([
#     transforms.RandomAffine(degrees=2, translate=(0.02, 0.02), scale=(0.98, 1.02), shear=2, fillcolor=(124,117,104)),
    transforms.Resize(size),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
#     transforms.RandomErasing(p=0.3, scale=(0.02, 0.10), ratio=(0.3, 3.3), value=0, inplace=True),
])
val_aug = transforms.Compose([
    transforms.Resize(size),
    transforms.ToTensor(),
])

# pin_memory : cuda pin memeory use
train_ = SiameseNetworkDataset(datasets.ImageFolder(train_dir), transform=train_aug, should_invert=False)
train_loader = DataLoader(train_, shuffle=True, num_workers=num_workers, batch_size=train_batch)
source_train_ = SiameseNetworkDataset(datasets.ImageFolder(source_train_dir), transform=val_aug, should_invert=False)
source_train_loader = DataLoader(source_train_, shuffle=True, num_workers=num_workers, batch_size=train_batch)
val_target_ = SiameseNetworkDataset(datasets.ImageFolder(val_target_dir), transform=val_aug, should_invert=False)
val_target_loader = DataLoader(val_target_, shuffle=True, num_workers=num_workers, batch_size=test_batch)
val_source_ = SiameseNetworkDataset(datasets.ImageFolder(val_source_dir), transform=val_aug, should_invert=False)
val_source_loader = DataLoader(val_source_, shuffle=True, num_workers=num_workers, batch_size=test_batch)

# Model

In [10]:
teacher_model = EfficientNet.from_name(model_name, num_classes=num_classes)
student_model = EfficientNet.from_name(model_name, num_classes=num_classes)

# Pre-trained
if pretrained:
    print("=> using pre-trained model '{}'".format(pretrained))
    pt = torch.load(pretrained)['state_dict']
    teacher_model.load_state_dict(pt)
    student_model.load_state_dict(pt)
#     pt.pop('_fc.weight')
#     pt.pop('_fc.bias')
#     for key, weights in pt.items():
#         student_model.state_dict()[key] = weights

=> using pre-trained model './log/style1/128/b1/checkpoint.pth.tar'


In [11]:
teacher_model.to('cuda')
student_model.to('cuda')
cudnn.benchmark = True
print('    Total params: %.2fM' % (sum(p.numel() for p in student_model.parameters())/1000000.0))

    Total params: 6.52M


In [12]:
for param in teacher_model.parameters():
    param.requires_grad = False
teacher_model.eval()

EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (_bn0): GroupNorm(8, 32, eps=1e-05, affine=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_bn1): GroupNorm(8, 32, eps=1e-05, affine=True)
      (_se_reduce): Conv2dStaticSamePadding(
        32, 8, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        8, 32, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False
        (static_padding): Identity()
      )
      (_bn2): GroupNorm(4, 16, ep

In [13]:
teacher_model_weights = {}
for name, param in teacher_model.named_parameters():
    teacher_model_weights[name] = param.detach()

In [14]:
def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

In [15]:
def reg_cls(model):
    l2_cls = torch.tensor(0.).cuda()
    for name, param in model.named_parameters():
        if name.startswith(fc_name):
            l2_cls += 0.5 * torch.norm(param) ** 2
    return l2_cls

def reg_l2sp(model):
    sp_loss = torch.tensor(0.).cuda()
    for name, param in model.named_parameters():
        if not name.startswith(fc_name):
            sp_loss += 0.5 * torch.norm(param - teacher_model_weights[name]) ** 2
    return sp_loss

# Loss

In [16]:
criterion = ContrastiveLoss(margin=1).cuda()
optimizer = optim.SGD(student_model.parameters(), lr=lr, momentum=momentum)
# optimizer = optim.Adam(model.parameters(), weight_decay=0)
scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
scheduler_warmup = GradualWarmupScheduler(optimizer, multiplier=4, total_epoch=50, after_scheduler=scheduler_cosine)

In [17]:
# Resume
if resume:
    print('==> Resuming from checkpoint..')
    checkpoint = os.path.dirname(resume)
#     checkpoint = torch.load(resume)
    resume = torch.load(resume)
    best_acc = resume['best_acc']
    start_epoch = resume['epoch']
    student_model.load_state_dict(resume['state_dict'])
    optimizer.load_state_dict(resume['optimizer'])
    logger = Logger(os.path.join(checkpoint, 'log.txt'), resume=True)
else:
    logger = Logger(os.path.join(checkpoint, 'log.txt'))
    logger.set_names(['Learning Rate', 'Train Loss', 'Valid Loss', 'Train Acc.', 'Valid Acc.', 'Source Loss', 'Source ACC.'])

# Train

In [18]:
def train(train_loader, source_train_loader, teacher_model, student_model, criterion, optimizer, epoch, use_cuda):
    student_model.train()
    torch.set_grad_enabled(True)
    
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    end = time.time()
    
#     source_set = []
#     for inputs0, inputs1, targets in source_train_loader:
#         source_set.append((inputs0, inputs1, targets))
    
    bar = Bar('Processing', max=len(train_loader))
    for batch_idx, (inputs0, inputs1, targets) in enumerate(train_loader):
        batch_size = inputs0.size(0)
        if batch_size < train_batch:
            continue
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs0, inputs1, targets = inputs0.cuda(), inputs1.cuda(), targets.cuda()
#         inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)
        
        # cutmix
#         source_inputs, source_targets = source_set[batch_idx]
#         source_inputs, source_targets = source_inputs.cuda(), source_targets.cuda()

#         prob_delta = cm_prob_init - cm_prob_low
#         prob_step = epoch / (epochs+1) * prob_delta
#         lam = cm_prob_init - prob_step

#         rand_index = torch.randperm(inputs.size()[0]).cuda()
#         st = source_targets[rand_index]
#         tt = targets[rand_index]
#         rand_index = rand_index[st == tt]

#         bbx1, bby1, bbx2, bby2 = rand_bbox(inputs.size(), lam)
#         inputs[rand_index, :, bbx1:bbx2, bby1:bby2] = source_inputs[rand_index, :, bbx1:bbx2, bby1:bby2]
#         lam = 1 - ((bbx2 - bbx1)*(bby2 - bby1) / (inputs.size()[-1] * inputs.size()[-2]))
        
        outputs0 = student_model(inputs0)
        outputs1 = student_model(inputs1)
        
        loss_main = criterion(outputs0, outputs1, targets)
        loss_cls = 0
        loss_sp = 0
        loss_cls = reg_cls(student_model)
        loss_sp = reg_l2sp(student_model)
            
            
        # compute output
        loss = loss_main + sp_alpha*loss_sp + sp_beta*loss_cls
        outputs = F.pairwise_distance(outputs0, outputs1, keepdim=True)

        # measure accuracy and record loss
        pred = outputs.data
        pred[pred < thresh] = 0.
        pred[pred >= thresh] = 1.
        prec1 = [accuracy_score(targets.data.cpu().numpy(), pred.cpu().numpy())]
        
#         prec1 = accuracy(outputs.data, targets.data)
        losses.update(loss.data.tolist(), inputs0.size(0))
        top1.update(prec1[0], inputs0.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        bar.suffix  = '({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Total: {total:} | ETA: {eta:} | Loss: {loss:.4f} | top1: {top1: .4f} '.format(
                    batch=batch_idx + 1,
                    size=len(train_loader),
                    data=data_time.val,
                    bt=batch_time.val,
                    total=bar.elapsed_td,
                    eta=bar.eta_td,
                    loss=losses.avg,
                    top1=top1.avg,
                    )
        bar.next()
#         if batch_idx % 10 == 0:
        print('{batch}/{size} Data:{data:.3f} | Batch:{bt:.3f} | Total:{total:} | ETA:{eta:} | Loss:{loss:} | top1:{tp1:}'.format(
                 batch=batch_idx+1, size=len(train_loader), data=data_time.val, bt=batch_time.val, total=bar.elapsed_td, eta=bar.eta_td, loss=losses.avg, tp1=top1.avg))
    bar.finish()
    return (losses.avg, top1.avg)

In [19]:
def test(val_loader, model, criterion, epoch, use_cuda):
    global best_acc

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()
    torch.set_grad_enabled(False)

    end = time.time()
    bar = Bar('Processing', max=len(val_loader))
    for batch_idx, (inputs0, inputs1, targets) in enumerate(val_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs0, inputs1, targets = inputs0.cuda(), inputs1.cuda(), targets.cuda()
#         inputs, targets = torch.autograd.Variable(inputs, volatile=True), torch.autograd.Variable(targets)

        # compute output
        outputs0 = model(inputs0)
        outputs1 = model(inputs1)
        loss_main = criterion(outputs0, outputs1, targets)
        loss_cls = 0
        loss_sp = 0
        loss_cls = reg_cls(model)
        loss_sp = reg_l2sp(model)
        loss = loss_main + sp_alpha*loss_sp + sp_beta*loss_cls
        outputs = F.pairwise_distance(outputs0, outputs1, keepdim=True)

        # measure accuracy and record loss
        pred = outputs.data
        pred[pred < thresh] = 0.
        pred[pred >= thresh] = 1.
        prec1 = [accuracy_score(targets.data.cpu().numpy(), pred.cpu().numpy())]
#         prec1 = accuracy(outputs.data, targets.data)
        losses.update(loss.data.tolist(), inputs0.size(0))
        top1.update(prec1[0], inputs0.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        bar.suffix  = '({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Total: {total:} | ETA: {eta:} | Loss: {loss:} | top1: {top1:}'.format(
                    batch=batch_idx + 1,
                    size=len(val_loader),
                    data=data_time.avg,
                    bt=batch_time.avg,
                    total=bar.elapsed_td,
                    eta=bar.eta_td,
                    loss=losses.avg,
                    top1=top1.avg,)
        bar.next()
    print('{batch}/{size} Data:{data:.3f} | Batch:{bt:.3f} | Total:{total:} | ETA:{eta:} | Loss:{loss:} | top1:{tp1:}'.format(
         batch=batch_idx+1, size=len(val_loader), data=data_time.val, bt=batch_time.val, total=bar.elapsed_td, eta=bar.eta_td, loss=losses.avg, tp1=top1.avg))
    bar.finish()
    return (losses.avg, top1.avg)

In [20]:
def save_checkpoint(state, is_best, checkpoint='checkpoint', filename='checkpoint.pth.tar'):
    filepath = os.path.join(checkpoint, filename)
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(checkpoint, 'model_best.pth.tar'))

def adjust_learning_rate(optimizer, epoch):
    global state
    lr_set = [1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5]
    lr_list = schedule.copy()
    lr_list.append(epoch)
    lr_list.sort()
    idx = lr_list.index(epoch)
    state['lr'] *= lr_set[idx]
    for param_group in optimizer.param_groups:
        param_group['lr'] = state['lr']

In [21]:
for epoch in range(start_epoch, epochs):
    # teacher feedback
    if epoch in iter_time:
        print("iterative training: feedback {}".format(epoch))
        teacher_model.load_state_dict(student_model.state_dict())
        teacher_model_weights = {}
        for name, param in teacher_model.named_parameters():
            teacher_model_weights[name] = param.detach()
    
    state['lr'] = optimizer.state_dict()['param_groups'][0]['lr']
    adjust_learning_rate(optimizer, epoch)
    print('\nEpoch: [%d | %d] LR: %f' % (epoch + 1, epochs, state['lr']))
    
    train_loss, train_acc = train(train_loader, source_train_loader, teacher_model, student_model, criterion, optimizer, epoch, use_cuda)
    
    if epoch % 20 == 0:
        test_loss, test_acc = test(val_target_loader, student_model, criterion, epoch, use_cuda)
        source_loss, source_acc = test(val_source_loader, student_model, criterion, epoch, use_cuda)

    
        logger.append([state['lr'], train_loss, test_loss, train_acc, test_acc, source_loss, source_acc])
        is_best = test_acc > best_acc
        best_acc = max(test_acc, best_acc)
        save_checkpoint({
            'epoch': epoch + 1,
            'state_dict' : student_model.state_dict(),
            'acc': test_acc,
            'best_acc': best_acc,
            'optimizer': optimizer.state_dict(),
        }, is_best, checkpoint=checkpoint)
    scheduler_warmup.step()


Epoch: [1 | 4000] LR: 0.010000
1/4 Data:1.593 | Batch:4.583 | Total:0:00:04 | ETA:0:00:14 | Loss:19.4808349609375 | top1:0.6
2/4 Data:0.008 | Batch:0.757 | Total:0:00:05 | ETA:0:00:06 | Loss:15.797689437866211 | top1:0.575
3/4 Data:0.013 | Batch:0.772 | Total:0:00:06 | ETA:0:00:03 | Loss:10.676998744408289 | top1:0.5833333333333334
4/4 Data:0.015 | Batch:0.940 | Total:0:00:07 | ETA:0:00:00 | Loss:8.171465270221233 | top1:0.56
161/161 Data:0.015 | Batch:0.218 | Total:0:01:10 | ETA:0:00:00 | Loss:0.9932899887316695 | top1:0.49906542056074765
39/39 Data:0.017 | Batch:0.417 | Total:0:00:19 | ETA:0:00:00 | Loss:0.9857310087252886 | top1:0.5065384615384615

Epoch: [2 | 4000] LR: 0.010600
1/4 Data:1.329 | Batch:2.276 | Total:0:00:02 | ETA:0:00:07 | Loss:0.85831218957901 | top1:0.54
2/4 Data:0.011 | Batch:0.710 | Total:0:00:02 | ETA:0:00:03 | Loss:1.0318330228328705 | top1:0.52
3/4 Data:0.013 | Batch:0.633 | Total:0:00:03 | ETA:0:00:02 | Loss:1.1984994212786357 | top1:0.5033333333333333
4/4 D

4/4 Data:0.015 | Batch:0.781 | Total:0:00:05 | ETA:0:00:00 | Loss:1.4354153275489807 | top1:0.53

Epoch: [20 | 4000] LR: 0.021400
1/4 Data:2.121 | Batch:3.369 | Total:0:00:03 | ETA:0:00:11 | Loss:1.327417254447937 | top1:0.47
2/4 Data:0.042 | Batch:0.911 | Total:0:00:04 | ETA:0:00:05 | Loss:1.2751346230506897 | top1:0.545
3/4 Data:0.016 | Batch:0.896 | Total:0:00:05 | ETA:0:00:02 | Loss:1.2683263222376506 | top1:0.53
4/4 Data:0.035 | Batch:0.947 | Total:0:00:06 | ETA:0:00:00 | Loss:1.2542507350444794 | top1:0.5075

Epoch: [21 | 4000] LR: 0.022000
1/4 Data:2.706 | Batch:3.601 | Total:0:00:03 | ETA:0:00:11 | Loss:1.0558252334594727 | top1:0.61
2/4 Data:0.008 | Batch:0.919 | Total:0:00:04 | ETA:0:00:05 | Loss:1.089461326599121 | top1:0.53
3/4 Data:0.008 | Batch:1.297 | Total:0:00:05 | ETA:0:00:02 | Loss:1.0871299902598064 | top1:0.51
4/4 Data:0.031 | Batch:0.902 | Total:0:00:06 | ETA:0:00:00 | Loss:1.0875824987888336 | top1:0.5025
161/161 Data:0.018 | Batch:0.318 | Total:0:01:36 | ETA:0:0

2/4 Data:0.024 | Batch:0.747 | Total:0:00:03 | ETA:0:00:04 | Loss:0.37111109495162964 | top1:0.53
3/4 Data:0.008 | Batch:0.582 | Total:0:00:04 | ETA:0:00:02 | Loss:0.3742988010247548 | top1:0.5233333333333333
4/4 Data:0.011 | Batch:0.598 | Total:0:00:04 | ETA:0:00:00 | Loss:0.3667387142777443 | top1:0.54

Epoch: [39 | 4000] LR: 0.032800
1/4 Data:1.673 | Batch:2.448 | Total:0:00:02 | ETA:0:00:08 | Loss:0.32340988516807556 | top1:0.59
2/4 Data:0.011 | Batch:0.659 | Total:0:00:03 | ETA:0:00:04 | Loss:0.34679172933101654 | top1:0.55
3/4 Data:0.023 | Batch:0.694 | Total:0:00:03 | ETA:0:00:02 | Loss:0.3601141571998596 | top1:0.5466666666666666
4/4 Data:0.011 | Batch:0.640 | Total:0:00:04 | ETA:0:00:00 | Loss:0.3481812998652458 | top1:0.5825

Epoch: [40 | 4000] LR: 0.033400
1/4 Data:1.949 | Batch:2.600 | Total:0:00:02 | ETA:0:00:08 | Loss:0.2658047378063202 | top1:0.74
2/4 Data:0.030 | Batch:0.784 | Total:0:00:03 | ETA:0:00:04 | Loss:0.31015588343143463 | top1:0.67
3/4 Data:0.012 | Batch:0.95

1/4 Data:2.348 | Batch:3.389 | Total:0:00:03 | ETA:0:00:11 | Loss:0.4977678954601288 | top1:0.58
2/4 Data:0.017 | Batch:1.040 | Total:0:00:04 | ETA:0:00:05 | Loss:0.48737452924251556 | top1:0.575
3/4 Data:0.026 | Batch:0.778 | Total:0:00:05 | ETA:0:00:02 | Loss:0.4837893843650818 | top1:0.5866666666666667
4/4 Data:0.019 | Batch:0.891 | Total:0:00:06 | ETA:0:00:00 | Loss:0.4756140038371086 | top1:0.5975

Epoch: [58 | 4000] LR: 0.040000
1/4 Data:2.583 | Batch:3.449 | Total:0:00:03 | ETA:0:00:11 | Loss:0.4049520492553711 | top1:0.64
2/4 Data:0.015 | Batch:1.032 | Total:0:00:04 | ETA:0:00:05 | Loss:0.3953447639942169 | top1:0.655
3/4 Data:0.011 | Batch:0.869 | Total:0:00:05 | ETA:0:00:02 | Loss:0.3926353454589844 | top1:0.6533333333333333
4/4 Data:0.022 | Batch:1.342 | Total:0:00:06 | ETA:0:00:00 | Loss:0.38364794105291367 | top1:0.655

Epoch: [59 | 4000] LR: 0.040000
1/4 Data:2.001 | Batch:2.811 | Total:0:00:02 | ETA:0:00:09 | Loss:0.4587520956993103 | top1:0.54
2/4 Data:0.015 | Batch:1.1

3/4 Data:0.008 | Batch:1.047 | Total:0:00:04 | ETA:0:00:02 | Loss:0.20194202661514282 | top1:0.8733333333333333
4/4 Data:0.009 | Batch:1.049 | Total:0:00:06 | ETA:0:00:00 | Loss:0.18981895223259926 | top1:0.8975

Epoch: [76 | 4000] LR: 0.039996
1/4 Data:1.877 | Batch:2.593 | Total:0:00:02 | ETA:0:00:08 | Loss:0.16112150251865387 | top1:0.95
2/4 Data:0.014 | Batch:0.982 | Total:0:00:03 | ETA:0:00:04 | Loss:0.17552249878644943 | top1:0.92
3/4 Data:0.045 | Batch:0.941 | Total:0:00:04 | ETA:0:00:02 | Loss:0.16893904904524484 | top1:0.94
4/4 Data:0.011 | Batch:1.082 | Total:0:00:05 | ETA:0:00:00 | Loss:0.17025191709399223 | top1:0.94

Epoch: [77 | 4000] LR: 0.039996
1/4 Data:2.147 | Batch:3.138 | Total:0:00:03 | ETA:0:00:10 | Loss:0.16350418329238892 | top1:0.96
2/4 Data:0.010 | Batch:0.983 | Total:0:00:04 | ETA:0:00:05 | Loss:0.17923150956630707 | top1:0.93
3/4 Data:0.010 | Batch:0.899 | Total:0:00:05 | ETA:0:00:02 | Loss:0.18283033867677054 | top1:0.92
4/4 Data:0.011 | Batch:1.090 | Total

...

1/4 Data:1.111 | Batch:1.718 | Total:0:00:01 | ETA:0:00:06 | Loss:0.0009179096668958664 | top1:1.0
2/4 Data:0.007 | Batch:0.625 | Total:0:00:02 | ETA:0:00:03 | Loss:0.0009648031555116177 | top1:1.0
3/4 Data:0.007 | Batch:0.622 | Total:0:00:02 | ETA:0:00:01 | Loss:0.0009632610405484835 | top1:1.0
4/4 Data:0.009 | Batch:0.632 | Total:0:00:03 | ETA:0:00:00 | Loss:0.00107898359419778 | top1:1.0

Epoch: [3912 | 4000] LR: 0.000000
1/4 Data:1.079 | Batch:1.710 | Total:0:00:01 | ETA:0:00:06 | Loss:0.0009528984082862735 | top1:1.0
2/4 Data:0.008 | Batch:0.586 | Total:0:00:02 | ETA:0:00:03 | Loss:0.0010339745203964412 | top1:1.0
3/4 Data:0.008 | Batch:0.481 | Total:0:00:02 | ETA:0:00:01 | Loss:0.001228203143303593 | top1:1.0
4/4 Data:0.009 | Batch:0.599 | Total:0:00:03 | ETA:0:00:00 | Loss:0.001599009265191853 | top1:0.9975

Epoch: [3913 | 4000] LR: 0.000000
1/4 Data:1.109 | Batch:1.699 | Total:0:00:01 | ETA:0:00:06 | Loss:0.0013115466572344303 | top1:1.0
2/4 Data:0.020 | Batch:0.621 | Total:0:0

2/4 Data:0.007 | Batch:0.623 | Total:0:00:02 | ETA:0:00:03 | Loss:0.0009773443453013897 | top1:1.0
3/4 Data:0.006 | Batch:0.614 | Total:0:00:03 | ETA:0:00:02 | Loss:0.0009714288947482904 | top1:1.0
4/4 Data:0.009 | Batch:0.639 | Total:0:00:03 | ETA:0:00:00 | Loss:0.0013466615928336978 | top1:1.0

Epoch: [3930 | 4000] LR: 0.000000
1/4 Data:1.111 | Batch:1.735 | Total:0:00:01 | ETA:0:00:06 | Loss:0.001339812413789332 | top1:1.0
2/4 Data:0.009 | Batch:0.637 | Total:0:00:02 | ETA:0:00:03 | Loss:0.0011864363332279027 | top1:1.0
3/4 Data:0.010 | Batch:0.605 | Total:0:00:02 | ETA:0:00:01 | Loss:0.0012200474739074707 | top1:1.0
4/4 Data:0.006 | Batch:0.619 | Total:0:00:03 | ETA:0:00:00 | Loss:0.0011633311223704368 | top1:1.0

Epoch: [3931 | 4000] LR: 0.000000
1/4 Data:1.165 | Batch:1.766 | Total:0:00:01 | ETA:0:00:06 | Loss:0.001148961833678186 | top1:1.0
2/4 Data:0.009 | Batch:0.594 | Total:0:00:02 | ETA:0:00:03 | Loss:0.006142006430309266 | top1:0.995
3/4 Data:0.020 | Batch:0.621 | Total:0:0

3/4 Data:0.006 | Batch:0.486 | Total:0:00:02 | ETA:0:00:01 | Loss:0.0015134429171060522 | top1:1.0
4/4 Data:0.010 | Batch:0.608 | Total:0:00:03 | ETA:0:00:00 | Loss:0.0014936972293071449 | top1:1.0

Epoch: [3948 | 4000] LR: 0.000000
1/4 Data:1.059 | Batch:1.690 | Total:0:00:01 | ETA:0:00:06 | Loss:0.0011400588555261493 | top1:1.0
2/4 Data:0.014 | Batch:0.646 | Total:0:00:02 | ETA:0:00:03 | Loss:0.0010834186687134206 | top1:1.0
3/4 Data:0.007 | Batch:0.611 | Total:0:00:02 | ETA:0:00:01 | Loss:0.0011578637640923262 | top1:1.0
4/4 Data:0.007 | Batch:0.652 | Total:0:00:03 | ETA:0:00:00 | Loss:0.0011493222264107317 | top1:1.0

Epoch: [3949 | 4000] LR: 0.000000
1/4 Data:1.094 | Batch:1.731 | Total:0:00:01 | ETA:0:00:06 | Loss:0.0010847083758562803 | top1:1.0
2/4 Data:0.010 | Batch:0.663 | Total:0:00:02 | ETA:0:00:03 | Loss:0.0011072054039686918 | top1:1.0
3/4 Data:0.010 | Batch:0.654 | Total:0:00:03 | ETA:0:00:02 | Loss:0.0010646504815667868 | top1:1.0
4/4 Data:0.010 | Batch:0.534 | Total:0:

1/4 Data:1.117 | Batch:1.730 | Total:0:00:01 | ETA:0:00:06 | Loss:0.0009286676649935544 | top1:1.0
2/4 Data:0.018 | Batch:0.614 | Total:0:00:02 | ETA:0:00:03 | Loss:0.0059448468091432005 | top1:0.995
3/4 Data:0.007 | Batch:0.606 | Total:0:00:02 | ETA:0:00:01 | Loss:0.004362354656526198 | top1:0.9966666666666667
4/4 Data:0.006 | Batch:0.622 | Total:0:00:03 | ETA:0:00:00 | Loss:0.003576021917979233 | top1:0.9975

Epoch: [3967 | 4000] LR: 0.000000
1/4 Data:1.102 | Batch:1.733 | Total:0:00:01 | ETA:0:00:06 | Loss:0.011705641634762287 | top1:0.99
2/4 Data:0.012 | Batch:0.608 | Total:0:00:02 | ETA:0:00:03 | Loss:0.006337796367006376 | top1:0.995
3/4 Data:0.008 | Batch:0.564 | Total:0:00:02 | ETA:0:00:01 | Loss:0.007409552131624271 | top1:0.9933333333333333
4/4 Data:0.006 | Batch:0.545 | Total:0:00:03 | ETA:0:00:00 | Loss:0.006834476851508953 | top1:0.995

Epoch: [3968 | 4000] LR: 0.000000
1/4 Data:1.153 | Batch:1.752 | Total:0:00:01 | ETA:0:00:06 | Loss:0.001131101162172854 | top1:1.0
2/4 Da

2/4 Data:0.009 | Batch:0.609 | Total:0:00:02 | ETA:0:00:03 | Loss:0.0014117067912593484 | top1:1.0
3/4 Data:0.008 | Batch:0.635 | Total:0:00:02 | ETA:0:00:01 | Loss:0.004114686589067181 | top1:0.9966666666666667
4/4 Data:0.008 | Batch:0.611 | Total:0:00:03 | ETA:0:00:00 | Loss:0.003358357062097639 | top1:0.9975

Epoch: [3985 | 4000] LR: 0.000000
1/4 Data:1.122 | Batch:1.636 | Total:0:00:01 | ETA:0:00:05 | Loss:0.009523876942694187 | top1:0.99
2/4 Data:0.007 | Batch:0.586 | Total:0:00:02 | ETA:0:00:03 | Loss:0.01106826402246952 | top1:0.99
3/4 Data:0.006 | Batch:0.635 | Total:0:00:02 | ETA:0:00:01 | Loss:0.007820864285652837 | top1:0.9933333333333333
4/4 Data:0.006 | Batch:0.615 | Total:0:00:03 | ETA:0:00:00 | Loss:0.008035149017814547 | top1:0.9925

Epoch: [3986 | 4000] LR: 0.000000
1/4 Data:1.154 | Batch:1.743 | Total:0:00:01 | ETA:0:00:06 | Loss:0.000934833602514118 | top1:1.0
2/4 Data:0.010 | Batch:0.633 | Total:0:00:02 | ETA:0:00:03 | Loss:0.00645879001240246 | top1:0.995
3/4 Data: